In [1]:
import numpy as np
from pathlib import Path
import os, json
import matplotlib.pyplot as plt
cwd = Path.cwd()
import UPPtoolbox as upp
import seaborn as sns
from tqdm.notebook import tqdm
import warnings


# Important variables
SNRs = np.array([-np.inf,-13,-11,-9,-7,-5,-3])
SubIDs = ['01','02','03','05','06','07','08','09','11','12','13','14','15','17','19','20','22','23','24','25']
colormap = {0: (0, 0, 0), 1: (0, 0.25, 1), 2: (0, 0.9375, 1), 3: (0, 0.91, 0.1), 4: (1, 0.6, 0), 5: (1, 0, 0), 6: (0.8, 0, 0)}

In [2]:
task = 'Active'
file_path = f'Results_EEG_v18/{task}/late'

all_params = []

test_save = []
    
for part in tqdm(range(20)):
    
    data_ref = f'myEpochs_{task}/Epoch_{SubIDs[part]}-epo.fif'
    epochs_file = cwd.parents[0] / data_ref
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        state_train = upp.STG(epochs_file, tmin=300, tmax=500)
        n_categories = len(list(state_train.keys()))
        categories = list(range(n_categories))

    # Input definition (same for all categories & trials)
    one_input = np.concatenate((np.zeros(75), np.ones(25), np.zeros(150)))
    input_train = {cat: np.stack([one_input for _ in range(state_train[cat].shape[0])]) for cat in categories}
    
    n_categories = len(list(state_train.keys()))
    categories = list(range(n_categories))

    # test
    linear_ukf = upp.model.StratifiedLinear(tau=10, process_noise=0.1, measure_noise=0.1)
    ll_ukf = linear_ukf.loglikelihood(state_train, input_train)
    ll_kf = linear_ukf.loglikelihood_kalman(state_train, input_train)
    print(ll_kf, ll_ukf)

#     # Cut some categories for a quicker test
#     state_train = {0: state_train[0], 1: state_train[4]}
#     input_train = {0: input_train[0], 1: input_train[4]}

#     # ---- Fit StratifiedLinear ----
#     linear_custom = upp.fitting_tools.clever_fit_linear(state_train = state_train, input_train = input_train, n_loops = 2, input_start_index=75, input_stop_index=100)
#     print('custom done')

#     linear_kf = upp.model.StratifiedLinear_kalman(tau=10, process_noise=0.1, measure_noise=0.1)
#     linear_kf.fit(
#         state_series=state_train,
#         input_series=input_train,
#         init_params=[10, 0.1, 0.1] + 7*[0],
#         bounds=[(1, 25), (0.01, 1), (0.01, 1)] + 7*[(0, 1)],
#         fixed_params = ['w0'] + [f'w{cat}' for cat in range(2, 7)])
#     print('kf done')

#     linear_ukf = upp.model.StratifiedLinear(tau=10, process_noise=0.1, measure_noise=0.1)
#     linear_ukf.fit(
#         state_series=state_train,
#         input_series=input_train,
#         init_params=[10, 0.1, 0.1] + 7*[0],
#         bounds=[(1, 25), (0.01, 1), (0.01, 1)] + 7*[(0, 1)],
#         fixed_params = ['w0'] + [f'w{cat}' for cat in range(2, 7)])
#     print('ukf done')

#     all_params.append([linear_custom.get_params(), linear_kf.get_params(), linear_ukf.get_params()])




# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # 1. Convert nested list to a Tidy DataFrame
# data_list = []
# methods = ['Custom', 'KF', 'UKF']

# for part_idx, results in enumerate(all_params):
#     for method_idx, params in enumerate(results):
#         # We flatten the dict and add metadata
#         row = params.copy()
#         row['Subject'] = SubIDs[part_idx]
#         row['Method'] = methods[method_idx]
#         data_list.append(row)

# df = pd.DataFrame(data_list)

# # 2. Visualization
# sns.set_theme(style="whitegrid")
# fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# # Parameters to plot
# params_to_show = ['tau', 'process_noise', 'measure_noise']
# titles = [r'$\tau$ (Time Constant)', 'Process Noise', 'Measurement Noise']

# for i, param in enumerate(params_to_show):
#     # Stripplot shows individual subjects, Boxplot shows distribution
#     sns.boxplot(data=df, x='Method', y=param, ax=axes[i], palette='Set2', width=0.5)
#     sns.stripplot(data=df, x='Method', y=param, ax=axes[i], color=".3", alpha=0.6)
#     axes[i].set_title(titles[i], fontweight='bold')
#     axes[i].set_xlabel('')
#     axes[i].set_ylabel('Estimated Value')

# plt.tight_layout()
# plt.show()

# # 3. Correlation Check (Validation: KF vs UKF)
# plt.figure(figsize=(6, 6))
# kf_vals = df[df['Method'] == 'KF']['tau'].values
# ukf_vals = df[df['Method'] == 'UKF']['tau'].values

# sns.scatterplot(x=kf_vals, y=ukf_vals)
# plt.plot([df['tau'].min(), df['tau'].max()], [df['tau'].min(), df['tau'].max()], 'r--')
# plt.title('Validation: KF vs UKF (Tau)')
# plt.xlabel('KF Estimate')
# plt.ylabel('UKF Estimate')
# plt.show()

  0%|          | 0/20 [00:00<?, ?it/s]

-2605936.3781553996 -2851809.446462482
-2725296.8429037924 -2958442.2290578173
-2381556.8874874073 -2611418.023005207
-2929169.0453032753 -3150722.4313039035
-2502422.871499689 -2709807.41931643
-3389384.5135732642 -3671456.995776483
-2648670.9208511002 -2868692.8989381082
-2767441.7880027215 -3004739.7567693926
-3415832.0718062576 -3727546.5919392295
-2172045.810976549 -2399670.547971413
-2207984.3989994684 -2450285.3075882643
-2614481.594212093 -2879050.8652732726
-2670700.4625357618 -2915984.804021208
-2701004.6933543063 -2949815.814749194
-3669614.242395213 -3974925.1220733044
-2943903.933909944 -3261216.4803527915
-2759082.078777688 -3036106.722252381
-3389652.633534797 -3724284.5617144722
-3101701.349956709 -3381027.7495853337
-2983571.8873727755 -3271305.1874002954


In [3]:
linear_ukf = upp.model.StratifiedLinear(tau=10, process_noise=0.1, measure_noise=0.1)
ll_ukf = linear_ukf.loglikelihood(state_train, input_train)
ll_kf = linear_ukf.loglikelihood_kalman(state_train, input_train)
ll_ukf_aug = linear_ukf.loglikelihood_ukf_augmented(state_train, input_train)
ll_kf_aug = linear_ukf.loglikelihood_kalman_augmented(state_train, input_train)
print(ll_kf, ll_kf_aug, ll_ukf, ll_ukf_aug)

AttributeError: 'StratifiedLinear' object has no attribute 'loglikelihood_ukf_augmented'

# draft

In [ ]:
# Choose a single trial and a fixed set of parameters
test_params = [10.0, 0.4, 0.6, 0.5] # tau, q, r, w

# Manually evaluate LL
linear_kf = upp.model.StratifiedLinear_kalman(tau=10, process_noise=0.1, measure_noise=0.1)
# linear_kf.set_params_from_list(test_params)
ll_kf = linear_kf.loglikelihood(state_train, input_train)

linear_ukf = upp.model.StratifiedLinear(tau=10, process_noise=0.1, measure_noise=0.1)
# linear_ukf.set_params_from_list(test_params)
ll_ukf = linear_ukf.loglikelihood(state_train, input_train)

print(f"KF Log-Likelihood: {ll_kf}")
print(f"UKF Log-Likelihood: {ll_ukf}")
print(f"Difference: {abs(ll_kf - ll_ukf)}")

In [ ]:
model = upp.model.StratifiedLinearAudit(tau=10, process_noise=0.4, measure_noise=0.6)
model.perform_audit(state_train, input_train)

In [ ]:
# linear_ukf.debug_ukf_step_by_step({0: state_train[0]}, {0: input_train[0]}, 0)
linear_ukf.debug_ukf_step_by_step(state_train[0][0], input_train[0][0], 0)